In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd

# Define Models 
def langmuir(C, K, Bmax):
    return Bmax * (K * C) / (1 + K * C)

def hill(C, Kd, n, Bmax):
    return Bmax * (C**n) / (Kd + C**n)

def davis(C, K, Bmax, offset):
    return Bmax * (K * C) / (1 + K * C) + offset

def r_squared(ydata, yfit):
    ss_res = np.sum((ydata - yfit) ** 2)
    ss_tot = np.sum((ydata - np.mean(ydata)) ** 2)
    return 1 - ss_res / ss_tot

# Sample Data
urea_conc = np.array([0, c1, c2, c3, c4])    # concentartion according to your system
binding_data = {
    300: [0, x1, x2, x3, x4],
    325: [0, x1, x2, x3, x4],
    350: [0, x1, x2, x3, x4],               # binding data, urea around protein
    375: [0, x1, x2, x3, x4],
}

colors = ['red', 'green', 'blue', 'purple']
C_fit = np.linspace(0, 1.3, 2)
model_results = {}


plt.figure(figsize=(10, 7))

# Fit & Plot Loop 
fit_rows = []
for i, (temp, color) in enumerate(zip(binding_data.keys(), colors)):
    ydata = np.array(binding_data[temp])
   
    # Fit all models
    popt_lang, _ = curve_fit(langmuir, urea_conc, ydata, p0=[0.01, max(ydata)])
    popt_hill, _ = curve_fit(hill, urea_conc, ydata, p0=[1.5, 0.1, max(ydata)])
    popt_davis, _ = curve_fit(davis, urea_conc, ydata, p0=[1.0, max(ydata), 1.0], maxfev=5000)

   
    # R²
    r2_lang = r_squared(ydata, langmuir(urea_conc, *popt_lang))
    r2_hill = r_squared(ydata, hill(urea_conc, *popt_hill))
    r2_davis = r_squared(ydata, davis(urea_conc, *popt_davis))

    # Store all results
    model_results[temp] = {
        'Langmuir': {'params': popt_lang, 'R²': r2_lang},
        'Hill': {'params': popt_hill, 'R²': r2_hill},
        'Davis': {'params': popt_davis, 'R²': r2_davis}
    }

    # Determine best model
    best_model = max(['Langmuir', 'Hill', 'Davis'], key=lambda m: model_results[temp][m]['R²'])

    # Fit curve for best model
    if best_model == 'Langmuir':
        fit_y = langmuir(C_fit, *popt_lang)
    elif best_model == 'Hill':
        fit_y = hill(C_fit, *popt_hill)
    else:
        fit_y = davis(C_fit, *popt_davis)

    # Plot
    plt.scatter(urea_conc, ydata, color=color, label=f'{temp} K Data')
    plt.plot(C_fit, fit_y, color=color, linestyle='-',
             label=f'{temp} K Best Fit: {best_model} (R²={model_results[temp][best_model]["R²"]:.3f})')

    # Save row for CSV
    params = model_results[temp][best_model]['params']
    row = {'Temperature': temp, 'Best_Model': best_model, 'R²': model_results[temp][best_model]['R²']}
    for j, p in enumerate(params):
        row[f'Param_{j+1}'] = p
    fit_rows.append(row)


plt.xlabel("Urea Concentration")
plt.ylabel("Average Bound Urea Molecules")

plt.legend(fontsize=9)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

# === Save Plot ===
plt.savefig("urea_binding_model_fits.png", dpi=300)
plt.show()

# === Save CSV ===
df = pd.DataFrame(fit_rows)
df.to_csv("best_model_fit_results.csv", index=False)
print("Saved plot to 'urea_binding_model_fits1.png'")
print("Saved fit data to 'best_model_fit_results.csv'")

# Print R² values for all models at each temperature
for temp in sorted(model_results):
    print(f"\nTemperature: {temp} K")
    for model in ['Langmuir', 'Hill', 'Davis']:
        r2 = model_results[temp][model]['R²']
        print(f"  {model} R²: {r2:.4f}")